In [1]:
import sys
sys.path.append('..') 
from src.vdb_managers.chroma_manager import ChromaManager

In [2]:
chroma_store = ChromaManager() # 自定义的chroma类

In [6]:
file_path = "files/论文 - GraphRAG.pdf"
# file_path = "files/上奇实习报告.pdf"
auto_extract_metadata = True
metadata = {
    'keywords': "实习", # 必须是str，好像bool等也行，不记得了，反正不能是list
    "from": "上齐实习"
}
collection_name = "RAG"
similarity_metric = "cosine"
dense_weight = 0.5
metadata_filter = {'关键字key': {'$in': ['graphrag', 'node', 'edge']}}

In [4]:
# 上传PDF文档
chroma_store.upload_pdf_file(file_path = file_path, collection_name = collection_name)
# chroma_store.upload_pdf_file(file_path = file_path, metadata=metadata, collection_name=collection_name)

In [7]:
query = "What are the general methods of deep reinforcement learning?"
k = 3

In [8]:
results = chroma_store.dense_search(collection_name=collection_name, query=query, k=k, metadata_filter = metadata_filter,fuzzy_filter=True)
print(results)

模糊元数据过滤后的过滤器： {'keywords': {'$in': ['Graph RAG', 'graph', 'graph nodes', 'graph indexes']}} 

{'ids': [['d4f37542-1406-4f94-ae39-b545ef0860ca', '5774ee75-5127-4928-9935-ea58aa07e305', 'c94159fd-4a56-459e-9eb5-089ded055c66']], 'embeddings': None, 'documents': [['NebulaGraph (2024). Nebulagraph launches industry-first graph rag: Retrieval-augmented genera-\ntion with llm based on knowledge graphs. https://www .nebula-graph .io/posts/graph-RAG.\nNeo4J (2024). Project NaLLM. https://github .com/neo4j/NaLLM.\nNewman, M. E. (2006). Modularity and community structure in networks. Proceedings of the\nnational academy of sciences , 103(23):8577–8582.\nRam, O., Levine, Y ., Dalmedigos, I., Muhlgay, D., Shashua, A., Leyton-Brown, K., and Shoham,\nY . (2023). In-context retrieval-augmented language models. Transactions of the Association for\nComputational Linguistics , 11:1316–1331.\nRanade, P. and Joshi, A. (2023). Fabula: Intelligence report generation using retrieval-augmented\nnarrative const

In [ ]:
results = chroma_store.hybrid_search(collection_name=collection_name, query=query, k=k, dense_weight=1)
results

In [ ]:
results = chroma_store.search(collection_name=collection_name, query=query, k=k, dense_weight=1)
results

In [ ]:
formatted_context = chroma_store.get_formatted_context(collection_name=collection_name, query=query, k=k, dense_weight=dense_weight)
formatted_context

### 2. help

In [6]:
import os
from dotenv import load_dotenv # type: ignore

load_dotenv()
api_key = os.getenv('ZETATECHS_API_KEY')
base_url = os.getenv('ZETATECHS_API_BASE')

import chromadb # type: ignore
import chromadb.utils.embedding_functions as embedding_functions # type: ignore

client = chromadb.PersistentClient(path="ChromaVDB")

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=api_key,
    api_base=base_url,
    model_name="text-embedding-3-large"
)

collection = client.get_or_create_collection(name="test", embedding_function=openai_ef)

In [ ]:
help(collection.query)

In [ ]:
help(client.get_or_create_collection)

In [ ]:
help(collection.add)

In [ ]:
help(client.create_collection)